# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [22]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [4]:
# Check current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# Join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(sorted(file_path_list)) #30 files as expected - from 11/1 to 11/30

/home/workspace
['/home/workspace/event_data/2018-11-01-events.csv', '/home/workspace/event_data/2018-11-02-events.csv', '/home/workspace/event_data/2018-11-03-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-07-events.csv', '/home/workspace/event_data/2018-11-08-events.csv', '/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-10-events.csv', '/home/workspace/event_data/2018-11-11-events.csv', '/home/workspace/event_data/2018-11-12-events.csv', '/home/workspace/event_data/2018-11-13-events.csv', '/home/workspace/event_data/2018-11-14-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-17-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-19-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [18]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
print('Total number of records in all 30 files ',len(full_data_rows_list))
#print(full_data_rows_list[0]) #there are multiple records in each file with multiple columns

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

emptyArtists = 0

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId']) #subsetting only a select few columns
    for row in full_data_rows_list:
        #print(row)
        #print(row[0]) #this is the artist name, so we are skipping events that do not involve an artist
        if (row[0] == ''): #skipping empty artists
            emptyArtists += 1
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

print('Total number of records w/ missing artists ',emptyArtists)
print('Total number of final records we will load to Cassandra ',len(full_data_rows_list)-emptyArtists)

Total number of records in all 30 files  8056
Total number of records w/ missing artists  1236
Total number of final records we will load to Cassandra  6820


In [19]:
# Check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f)) #these are the non-missing artists and the numbers add up; header is the additional one record i think

6821


In [2]:
# Taking a look at the CSV/events we will load
df = pd.read_csv('event_datafile_new.csv')
df.head()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Mohammad,M,0,Rodriguez,277.15873,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Horn Concerto No. 4 in E flat K495: II. Romanc...,88
1,Jimi Hendrix,Mohammad,M,1,Rodriguez,239.82975,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Woodstock Inprovisation,88
2,Building 429,Mohammad,M,2,Rodriguez,300.61669,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Majesty (LP Version),88
3,The B-52's,Gianna,F,0,Jones,321.54077,free,"New York-Newark-Jersey City, NY-NJ-PA",107,Love Shack,38
4,Die Mooskirchner,Gianna,F,1,Jones,169.29914,free,"New York-Newark-Jersey City, NY-NJ-PA",107,Frisch und g'sund,38


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now we are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data appears like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [23]:
# Making a connection to a Cassandra instance in the local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [24]:
#Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS events_project
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
    """)

except Exception as e:
    print(e)

#### Set Keyspace

In [25]:
#Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('events_project')
except Exception as e:
    print(e)

## Creating queries to answer the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

### Query 1

#### Create the table to run the query 'Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4'

The primary key for this table is a composite key - sessionId and itemInSession. Using this combination ensures we get a unique record since there can be multiple items in a given session and since we are querying on both the columns, we are using both of them as partitioning keys.

In [26]:
try:
    session.execute("""
        DROP TABLE IF EXISTS sessionId_and_itemInSession
    """)

except Exception as e:
    print(e)
    
try:
    session.execute("""
        CREATE TABLE IF NOT EXISTS sessionId_and_itemInSession (
            sessionId INT,
            itemInSession INT,
            artist VARCHAR,
            song VARCHAR,
            length FLOAT,
            PRIMARY KEY ((sessionId, itemInSession))); 
    """)

except Exception as e:
    print(e)

#### Insert data into the table above

In [27]:
#Insert data into the table above
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO sessionId_and_itemInSession (sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5]))) #These data types are not implicitly converted, so need to cast them here

#### Run the SELECT query to verify that the data have been inserted into each table

In [28]:
try:
    results = session.execute("""
        SELECT 
            artist, 
            song, 
            length
        FROM sessionId_and_itemInSession
        WHERE sessionId = 338 
            AND itemInSession = 4
    """)
    
except Exception as e:
    print(e)

for result in results:
    print ('Artist: ',result.artist, 'Song: ',result.song, 'Length: ',result.length)

Artist:  Faithless Song:  Music Matters (Mark Knight Dub) Length:  495.30731201171875


### Query 2

#### Create the table to run the query 'Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182'

The primary key for this table is a composite key - userId and sessionId as partitioning keys and itemInSession as a clustering key. We need to include userId and sessionId as partitioning keys since we are querying by those columns and we need to use itemInSession as a clustering key to help with the sorting requirement. These three columns also uniquely identify a record.

In [29]:
try:
    session.execute("""
        DROP TABLE IF EXISTS userId_and_sessionId
    """)

except Exception as e:
    print(e)
    
try:
    session.execute("""
        CREATE TABLE IF NOT EXISTS userId_and_sessionId (
            userId INT,
            sessionId INT,
            itemInSession INT,
            artist VARCHAR,
            song VARCHAR,
            firstName VARCHAR,
            lastName VARCHAR,
            PRIMARY KEY ((userId, sessionId), itemInSession));
    """)

except Exception as e:
    print(e)

#### Insert data into the table above

In [30]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) #skip header
    for line in csvreader:
        query = "INSERT INTO userId_and_sessionId (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s);"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

#### Run the SELECT query to verify that the data have been inserted into each table

In [31]:
try:
    results = session.execute("""
        SELECT 
            artist, 
            song, 
            firstName,
            lastName
        FROM userId_and_sessionId
        WHERE userId = 10 
            AND sessionId = 182
        ORDER BY itemInSession
    """)
    
except Exception as e:
    print(e)

for result in results:
    print ('Artist: ',result.artist, 'Song: ',result.song, 'First Name: ',result.firstname, 'Last Name: ',result.lastname)

Artist:  Down To The Bone Song:  Keep On Keepin' On First Name:  Sylvie Last Name:  Cruz
Artist:  Three Drives Song:  Greece 2000 First Name:  Sylvie Last Name:  Cruz
Artist:  Sebastien Tellier Song:  Kilometer First Name:  Sylvie Last Name:  Cruz
Artist:  Lonnie Gordon Song:  Catch You Baby (Steve Pitron & Max Sanna Radio Edit) First Name:  Sylvie Last Name:  Cruz


### Query 3

#### Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In this table, we are using song (partition key) and userId (clustering key) as our primary key combination. If we do not use userId, we will remove instances where multiple users listen to the same song, so using userId in the primary key helps us maintain each record as unique.

In [32]:
try:
    session.execute("""
        DROP TABLE IF EXISTS name_and_song
    """)

except Exception as e:
    print(e)
    
try:
    session.execute("""
        CREATE TABLE IF NOT EXISTS name_and_song (
            song VARCHAR,
            userId INT,
            firstName VARCHAR,
            lastName VARCHAR,
            PRIMARY KEY (song, userId)); 
    """)

except Exception as e:
    print(e)        

#### Insert data into the table above

In [33]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) #skip header
    for line in csvreader:
        query = "INSERT INTO name_and_song (song, userId, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s);"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

#### Run the SELECT query to verify that the data have been inserted into each table

In [34]:
try:
    results = session.execute("""
        SELECT 
            firstName,
            lastName
        FROM name_and_song
        WHERE song = 'All Hands Against His Own'
    """)
    
except Exception as e:
    print(e)

for result in results:
    print ('First Name: ',result.firstname, 'Last Name: ',result.lastname)

First Name:  Jacqueline Last Name:  Lynch
First Name:  Tegan Last Name:  Levine
First Name:  Sara Last Name:  Johnson


### Drop the tables before closing out the sessions

In [16]:
try:
    session.execute("DROP TABLE IF EXISTS sessionId_and_itemInSession")
    session.execute("DROP TABLE IF EXISTS userId_and_sessionId")
    session.execute("DROP TABLE IF EXISTS name_and_song")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [17]:
session.shutdown()
cluster.shutdown()